In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Gurneet Singh\\Desktop\\projects\\chicken-disease\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Gurneet Singh\\Desktop\\projects\\chicken-disease'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)         # Define Entity for Custom Data Type
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,     #Returns BoxType 
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:         # Custom Return Type using Entity
        config = self.config.data_ingestion         # Get Configuration from Config.yamlAAAAAAAAAAAAZZZ
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Args:
            url: str
            Download the file from the given URL
        Returns:
            None (only logs the file download status)
        """
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} Downloaded!! With the Following Header: {header}")
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        """
        Arg:
            zip_file_path: str
            Extracts the zip file into the data directoruy
        Returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [11]:
try:            # Pipeline Code to Run the Data Ingestion Process
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-16 00:13:13,926: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-16 00:13:13,934: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-16 00:13:13,936: INFO: common: Directory created: artifacts]
[2024-06-16 00:13:13,937: INFO: common: Directory created: artifacts/data_ingestion]
[2024-06-16 00:13:21,268: INFO: 420735829: artifacts/data_ingestion/data.zip Downloaded!! With the Following Header: Connection: close
Content-Length: 15828590
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "83fddb1868a9869e589886922f8489a09984811f7d9a9cd9b45a21969993c79e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8A64:12D333:147725:1B0347:666DE0C8
Accept-Ranges: bytes
Date: Sat, 15 Jun 2024 18:43:22 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10249-MAA
X-Cache: